# WAYLONGO CODE
https://github.com/waylongo/cis-challenge-energy-prediction/blob/main/notebooks/modelling-lgb.ipynb

reference just using this so can compare their resulting test value with mine that used HPOAs

used this code also as a reference for my own lgbm model

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from datetime import datetime
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 100)

In [5]:
df_train = pd.read_pickle("../../Data/Prepared_Data/supervised/consumption_all.pkl")
display(df_train)
df_test = pd.read_pickle("../../Data/Prepared_Data/supervised/consumption_all_preds.pkl")
display(df_test)

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,1.0,1,0,2.0,terraced_house,0.0,0.0,0.0,1.0,3.5170
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,1.0,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,2.9330
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,1.0,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,3.0850
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,1.0,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,3.2370
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,1.0,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,4.4710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640294,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,1.0,0,2,3.0,detached_house,1.0,0.0,0.0,0.0,18.7585
640295,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,1.0,0,2,3.0,detached_house,1.0,0.0,0.0,0.0,21.0110
640296,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,1.0,0,2,3.0,detached_house,1.0,0.0,0.0,0.0,19.2040
640297,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,1.0,1,2,3.0,detached_house,1.0,0.0,0.0,0.0,17.3970


,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,month_cos,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,NaN
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,NaN
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,NaN
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,NaN
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0.841254,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181865,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,0.841254,0,2,3.0,detached_house,1.0,0.0,0.0,0.0,NaN
1181866,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,0.841254,0,2,3.0,detached_house,1.0,0.0,0.0,0.0,NaN
1181867,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,0.841254,1,2,3.0,detached_house,1.0,0.0,0.0,0.0,NaN
1181868,0xcd19e6fe3d887bc5dcac7ca18d46199695463fdd,0,1,1,0,0,1,0,1,0,...,0.841254,1,2,3.0,detached_house,1.0,0.0,0.0,0.0,NaN


In [6]:

le = preprocessing.LabelEncoder()
le.fit(df_test["meter_id"])

df_train["meter_id_code"] = le.transform(df_train["meter_id"])
df_test["meter_id_code"] = le.transform(df_test["meter_id"])

df_train.head()

,meter_id,meter_id_0,meter_id_1,meter_id_2,meter_id_3,meter_id_4,meter_id_5,meter_id_6,meter_id_7,meter_id_8,...,is_weekend,energy_cluster,num_bedrooms,dwelling_type,detached,flat,semi_detached,terraced,meter_reading,meter_id_code
0,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,1,0,2.0,terraced_house,0.0,0.0,0.0,1.0,3.517,2067
1,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,2.933,2067
2,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,3.085,2067
3,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,3.237,2067
4,0xa62b9f23553ff183f61e2bf943aab3d5983d02d7,0,0,0,0,0,0,0,0,0,...,0,0,2.0,terraced_house,0.0,0.0,0.0,1.0,4.471,2067


In [9]:
TARGET = "meter_reading"

#using different columns as we didn't use meter_reading_mean but rest are similar
use_cols = ["meter_id_code", "day_of_week", "day_of_month", "month_ord", "month_sin", "month_cos"]
print(f"used cols are: {use_cols}")

cat_cols = ["meter_id_code", "day_of_week", "day_of_month", "month_ord"]
print(f"categorical cols are: {cat_cols}")

for col in cat_cols:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')

used cols are: ['meter_id_code', 'day_of_week', 'day_of_month', 'month_ord', 'month_sin', 'month_cos']
categorical cols are: ['meter_id_code', 'day_of_week', 'day_of_month', 'month_ord']


In [10]:
param = {
    'metric': 'mae',
    'num_leaves': 256,
    'learning_rate': 0.01,
    'n_estimators': 10000,
    'subsample': 0.85, 
    "lambda_l1": 16,
    "lambda_l2": 8,
    "max_depth":8,
    'seed': 42,
    'n_jobs':-1
}

In [16]:
cv = 3
kf = KFold(n_splits=cv, shuffle=True, random_state=42)
skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

for train_index, valid_index in skf.split(df_train, df_train["meter_id"]):

    X_train, X_valid = df_train.loc[train_index, use_cols], df_train.loc[valid_index, use_cols]
    y_train, y_valid = df_train.loc[train_index, TARGET], df_train.loc[valid_index, TARGET]

    model_lgb = lgb.LGBMRegressor(**param)
    model_lgb.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        categorical_feature=cat_cols,
        early_stopping_rounds=200,
        verbose=1000)

    oof_valid = model_lgb.predict(X_valid, num_iteration=model_lgb.best_iteration_)
    df_train.loc[valid_index, "oof"] = oof_valid
    
    mae = np.sqrt(mean_absolute_error(y_valid, oof_valid))
    
    #TEST
    df_test[TARGET] += model_lgb.predict(df_test[use_cols], num_iteration=model_lgb.best_iteration_) / cv

df_test["meter_reading"] = df_test.meter_reading.clip(lower=0)
oof_mae = mean_absolute_error(df_train[TARGET], df_train["oof"])
print("#" * 50)
print(f"OOF MAE SCORE IS: {oof_mae:0.3f}") #1.362

[LightGBM] [Warning] lambda_l1 is set=16, reg_alpha=0.0 will be ignored. Current value: lambda_l1=16
[LightGBM] [Warning] lambda_l2 is set=8, reg_lambda=0.0 will be ignored. Current value: lambda_l2=8
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's l1: 2.47353
[2000]	valid_0's l1: 2.16051
[3000]	valid_0's l1: 2.05344
[4000]	valid_0's l1: 1.99646


KeyboardInterrupt: 

In [15]:
valid_score=mean_absolute_error(df_train[TARGET], df_train["oof"]) 
valid_score

1.886832527406821

In [14]:
# Write to submission
df_month = df_test.groupby(["meter_id", "month_ord"])["meter_reading"].sum().unstack()

month_cols = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
sub = pd.DataFrame(columns=["meter_id"] + month_cols)

sub["meter_id"] = df_month.index.values
sub[month_cols] = df_month.values.round()

sub.to_csv("./lgb_baseline_clean.csv", index=False, header=True)